In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer(language="english")
from nltk.stem.porter import PorterStemmer
stemmer2 = PorterStemmer()
from nltk.stem.lancaster import LancasterStemmer
stemmer3 = LancasterStemmer()
import string
punct = string.punctuation
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


In [2]:
train_tweets = pd.read_csv("C:/mini project/Coronavirus tweets - Text Classification/Corona_NLP_train.csv", encoding="latin1")
test_tweets = pd.read_csv("C:/mini project/Coronavirus tweets - Text Classification/Corona_NLP_test.csv", encoding="latin1")

In [3]:
train_tweets.head()


,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative


In [4]:
train_tweets.drop(["UserName" , "ScreenName" , "Location"] , axis = 1 , inplace = True)
test_tweets.drop(["UserName" , "ScreenName" , "Location"], axis = 1 , inplace = True)

In [5]:
print(train_tweets.info())
print(test_tweets.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41157 entries, 0 to 41156
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   TweetAt        41157 non-null  object
 1   OriginalTweet  41157 non-null  object
 2   Sentiment      41157 non-null  object
dtypes: object(3)
memory usage: 964.7+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3798 entries, 0 to 3797
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   TweetAt        3798 non-null   object
 1   OriginalTweet  3798 non-null   object
 2   Sentiment      3798 non-null   object
dtypes: object(3)
memory usage: 89.1+ KB
None


In [6]:
train_tweets["Sentiment"].value_counts()



Positive              11422
Negative               9917
Neutral                7713
Extremely Positive     6624
Extremely Negative     5481
Name: Sentiment, dtype: int64

In [7]:
from category_encoders import OrdinalEncoder
maplist = [{'col': 'Sentiment', 'mapping': {'Extremely Negative': 0, 'Negative': 1,'Neutral': 2, 'Positive': 3, 'Extremely Positive': 4}}]
enc = OrdinalEncoder(mapping=maplist)
train_tweets = enc.fit_transform(train_tweets)
test_tweets = enc.transform(test_tweets)

In [8]:
Ytrain = train_tweets["Sentiment"]
Ytest = test_tweets["Sentiment"]
Xtrain = train_tweets.drop("Sentiment" , axis = 1)
Xtest = test_tweets.drop("Sentiment" , axis = 1)


In [9]:
Xtrain


,TweetAt,OriginalTweet
0,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...
1,16-03-2020,advice Talk to your neighbours family to excha...
2,16-03-2020,Coronavirus Australia: Woolworths to give elde...
3,16-03-2020,My food stock is not the only one which is emp...
4,16-03-2020,"Me, ready to go at supermarket during the #COV..."
...,...,...
41152,14-04-2020,Airline pilots offering to stock supermarket s...
41153,14-04-2020,Response to complaint not provided citing COVI...
41154,14-04-2020,You know itÂs getting tough when @KameronWild...
41155,14-04-2020,Is it wrong that the smell of hand sanitizer i...


In [10]:
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()

In [11]:
# Encode labels in column 'species'.
Xtrain['TweetAt']= label_encoder.fit_transform(Xtrain['TweetAt'])
Xtest['TweetAt']= label_encoder.fit_transform(Xtest['TweetAt'])
Xtrain


,TweetAt,OriginalTweet
0,14,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...
1,14,advice Talk to your neighbours family to excha...
2,14,Coronavirus Australia: Woolworths to give elde...
3,14,My food stock is not the only one which is emp...
4,14,"Me, ready to go at supermarket during the #COV..."
...,...,...
41152,13,Airline pilots offering to stock supermarket s...
41153,13,Response to complaint not provided citing COVI...
41154,13,You know itÂs getting tough when @KameronWild...
41155,13,Is it wrong that the smell of hand sanitizer i...


In [12]:
stop_words= stopwords.words("english")
stop_words.remove("very")
stop_words.remove( "so")

In [13]:
def process(tweet) : 
    tweet = tweet.lower()
    tweet = re.sub(r'https?://[^\s\n\r]+', '', tweet)
    tweet = re.sub("@[A-Za-z0-9_]+","", tweet)
    for p in punct :
        while(str(p) in tweet ) :
            tweet = tweet.replace(p , "")
    tweet = word_tokenize(tweet)
    tweet = [w for w in tweet if (w not in stop_words )]
    entier= ['0','1','2','3','4','5','6','7','8','9']
    
    tweet2 = tweet.copy()
    for i in tweet2 : 
        for j in i : 
            if j in entier : 
                tweet.remove(i)
                break

    #tweet = [stemmer.stem(w) for w in tweet]
    tweet =" ".join(tweet)
    return tweet

In [14]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer


In [15]:
vectorizer = CountVectorizer(max_features=5000)
bow_train = vectorizer.fit_transform(Xtrain["OriginalTweet"])
features = vectorizer.get_feature_names_out()
Xtrain_bow=pd.DataFrame(bow_train.toarray(), columns = features)

In [16]:
bow_test = vectorizer.transform(Xtest["OriginalTweet"])
Xtest_bow=pd.DataFrame(bow_test.toarray(), columns = features)
Xtest_bow

,00,000,00pm,03,08,09,10,100,1000,10am,...,youâ,yoy,yr,yâ,zealand,zero,zimbabwe,zombie,zone,zoom
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3793,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3794,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3795,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3796,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
Xtrain_bow["love"].value_counts()


0    40884
1      265
2        8
Name: love, dtype: int64

In [18]:
Xtrain_bow  = pd.concat([Xtrain["TweetAt"], Xtrain_bow] , axis =1)
Xtest_bow  = pd.concat([Xtest["TweetAt"], Xtest_bow] , axis =1)

In [19]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
from sklearn.svm import SVC
svm = SVC()

In [20]:
#naive bayes
nb.fit(Xtrain_bow, Ytrain)
ypred_tr = nb.predict(Xtrain_bow)
ypred_ts = nb.predict(Xtest_bow)
print("Training Results:\n")
print(classification_report(Ytrain, ypred_tr))

Training Results:

              precision    recall  f1-score   support

           0       0.61      0.66      0.63      5481
           1       0.57      0.50      0.53      9917
           2       0.55      0.71      0.62      7713
           3       0.59      0.47      0.52     11422
           4       0.60      0.67      0.63      6624

    accuracy                           0.58     41157
   macro avg       0.58      0.60      0.59     41157
weighted avg       0.58      0.58      0.58     41157



In [21]:
print("Test Results:\n")
ypred_ts = nb.predict(Xtest_bow)
print(classification_report(Ytest, ypred_ts))

Test Results:

              precision    recall  f1-score   support

           0       0.46      0.61      0.52       592
           1       0.47      0.40      0.43      1041
           2       0.58      0.61      0.59       619
           3       0.46      0.41      0.43       947
           4       0.54      0.57      0.55       599

    accuracy                           0.50      3798
   macro avg       0.50      0.52      0.51      3798
weighted avg       0.49      0.50      0.49      3798



In [22]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

vocab_size = 20000
max_length = 60
embedding_dim = 64
trunc_type='post'
oov_tok = "<OOV>"


# Initialize the Tokenizer class
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)

# Generate the word index dictionary for the training sentences
tokenizer.fit_on_texts(Xtrain["OriginalTweet"])
word_index = tokenizer.word_index

# Generate and pad the training sequences
sequences = tokenizer.texts_to_sequences(Xtrain["OriginalTweet"])
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)

# Generate and pad the test sequences
testing_sequences = tokenizer.texts_to_sequences(Xtest["OriginalTweet"])
testing_padded = pad_sequences(testing_sequences,maxlen=max_length, truncating=trunc_type)

In [23]:
import tensorflow as tf



In [24]:
# Build the model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 64, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])

# Setup the training parameters
model.compile(loss='sparse_categorical_crossentropy',optimizer="adam",metrics=['accuracy'])

# Print the model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 60, 64)            1280000   
                                                                 
 flatten (Flatten)           (None, 3840)              0         
                                                                 
 dropout (Dropout)           (None, 3840)              0         
                                                                 
 dense (Dense)               (None, 16)                61456     
                                                                 
 dense_1 (Dense)             (None, 5)                 85        
                                                                 
Total params: 1,341,541
Trainable params: 1,341,541
Non-trainable params: 0
_________________________________________________________________


In [25]:
num_epochs = 30

# Train the model
model.fit(padded, Ytrain, epochs=num_epochs, validation_data=(testing_padded, Ytest))

Epoch 1/30
1287/1287 [==============================] - 37s 27ms/step - loss: 1.2760 - accuracy: 0.4435 - val_loss: 1.0525 - val_accuracy: 0.5811
Epoch 2/30
1287/1287 [==============================] - 36s 28ms/step - loss: 0.8015 - accuracy: 0.6936 - val_loss: 0.9402 - val_accuracy: 0.6348
Epoch 3/30
1287/1287 [==============================] - 35s 27ms/step - loss: 0.5621 - accuracy: 0.7978 - val_loss: 0.9950 - val_accuracy: 0.6393
Epoch 4/30
1287/1287 [==============================] - 36s 28ms/step - loss: 0.3983 - accuracy: 0.8629 - val_loss: 1.0378 - val_accuracy: 0.6382
Epoch 5/30
1287/1287 [==============================] - 36s 28ms/step - loss: 0.2977 - accuracy: 0.8953 - val_loss: 1.1193 - val_accuracy: 0.6316
Epoch 6/30
1287/1287 [==============================] - 43s 33ms/step - loss: 0.2362 - accuracy: 0.9179 - val_loss: 1.2744 - val_accuracy: 0.6209
Epoch 7/30
1287/1287 [==============================] - 42s 33ms/step - loss: 0.1948 - accuracy: 0.9311 - val_loss: 1.3298 -

In [26]:
#lstm
embedding_vector_features=64
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dropout, Embedding, LSTM , Dense
model=Sequential()

model.add(Embedding(vocab_size,embedding_vector_features,input_length=max_length))

model.add(LSTM(64,activation='relu',return_sequences=True))

model.add(Dropout(0.4))

model.add(LSTM(64,activation='relu'))

model.add(Dropout(0.4))
model.add(Dense(5,activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 60, 64)            1280000   
                                                                 
 lstm (LSTM)                 (None, 60, 64)            33024     
                                                                 
 dropout_1 (Dropout)         (None, 60, 64)            0         
                                                                 
 lstm_1 (LSTM)               (None, 64)                33024     
                                                                 
 dropout_2 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 5)                 325       
                                                                 
Total params: 1,346,373
Trainable params: 1,346,373
No

In [27]:
model.fit(padded, Ytrain, epochs=20, validation_data=(testing_padded, Ytest))


Epoch 1/20
1287/1287 [==============================] - 121s 90ms/step - loss: 1.3070 - accuracy: 0.4379 - val_loss: 0.9473 - val_accuracy: 0.6216
Epoch 2/20
1287/1287 [==============================] - 112s 87ms/step - loss: 0.9856 - accuracy: 0.6860 - val_loss: 0.7831 - val_accuracy: 0.7180
Epoch 3/20
1287/1287 [==============================] - 113s 88ms/step - loss: 0.6103 - accuracy: 0.7937 - val_loss: 0.7139 - val_accuracy: 0.7522
Epoch 4/20
1287/1287 [==============================] - 117s 91ms/step - loss: 0.5188 - accuracy: 0.8295 - val_loss: 0.7349 - val_accuracy: 0.7504
Epoch 5/20
1287/1287 [==============================] - 117s 91ms/step - loss: 0.4554 - accuracy: 0.8502 - val_loss: 0.7378 - val_accuracy: 0.7464
Epoch 6/20
1287/1287 [==============================] - 115s 89ms/step - loss: 0.4313 - accuracy: 0.8601 - val_loss: 0.7459 - val_accuracy: 0.7617
Epoch 7/20
1287/1287 [==============================] - 117s 91ms/step - loss: 0.4460 - accuracy: 0.8652 - val_loss: 0

In [33]:
>>> import nltk
>>> nltk.download('punkt')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Sachin\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [37]:
to_test = tokenizer.texts_to_sequences(np.array([process(" i am happy")  ]))
testing_tweet = pad_sequences(to_test,maxlen=max_length, truncating=trunc_type)

In [39]:
model.predict(testing_tweet)


1/1 [==============================] - 1s 595ms/step


array([[4.6074633e-09, 1.7133463e-05, 2.6345463e-03, 9.9311793e-01,
        4.2303759e-03]], dtype=float32)